In [12]:
%load_ext autoreload
%autoreload 2

import sys 
import os
sys.path.append('/home/azhar04/project/1. dev/quantum-dev/ShotOptimized-ADAPT-VQE/')

from openfermion.transforms import jordan_wigner
from openfermion.utils import commutator

from src.pools import SD, GSD, GSD1, SingletGSD, SpinCompGSD, PauliPool,  NoZPauliPool1, NoZPauliPool, QE, QE1, QE_All, CEO, OVP_CEO, DVG_CEO, DVE_CEO, MVP_CEO
from src.molecules import create_h2
from src.utilities import to_qiskit_operator

# Molecule
R = 0.742
molecule = create_h2(R)
molecule.description

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


'H2'

In [18]:
H = molecule.get_molecular_hamiltonian()
Hq = jordan_wigner(H)
Hq

-0.0996319255760924 [] +
-0.04533060139114829 [X0 X1 Y2 Y3] +
0.04533060139114829 [X0 Y1 Y2 X3] +
0.04533060139114829 [Y0 X1 X2 Y3] +
-0.04533060139114829 [Y0 Y1 X2 X3] +
0.17110568429296397 [Z0] +
0.1685935681755208 [Z0 Z1] +
0.1205103673498828 [Z0 Z2] +
0.1658409687410311 [Z0 Z3] +
0.17110568429296397 [Z1] +
0.1658409687410311 [Z1 Z2] +
0.1205103673498828 [Z1 Z3] +
-0.22250984629381015 [Z2] +
0.1743208433444256 [Z2 Z3] +
-0.22250984629381015 [Z3]

### Molecular Hamiltonian

In [20]:
Hqiskit = to_qiskit_operator(Hq)
Hqiskit_commuted = Hqiskit.group_commuting(qubit_wise=True)
Hqiskit_commuted

[SparsePauliOp(['YYXX', 'IIII'],
               coeffs=[-0.0453306 +0.j, -0.09963193+0.j]),
 SparsePauliOp(['XYYX'],
               coeffs=[0.0453306+0.j]),
 SparsePauliOp(['YXXY'],
               coeffs=[0.0453306+0.j]),
 SparsePauliOp(['XXYY'],
               coeffs=[-0.0453306+0.j]),
 SparsePauliOp(['IIIZ', 'IIZI', 'IZII', 'ZIII', 'IIZZ', 'IZIZ', 'ZIIZ', 'IZZI', 'ZIZI', 'ZZII'],
               coeffs=[ 0.17110568+0.j,  0.17110568+0.j, -0.22250985+0.j, -0.22250985+0.j,
   0.16859357+0.j,  0.12051037+0.j,  0.16584097+0.j,  0.16584097+0.j,
   0.12051037+0.j,  0.17432084+0.j])]

### Fermionic GSD Pool

In [30]:
gsd_pool = GSD1(molecule)
gsd_pool.__dict__


# for i in range(len(gsd_pool.operators)):
#     print(f'Fermionic Operator-{i}: {gsd_pool.operators[i]._f_operator}')
# print('')

for i in range(len(gsd_pool.operators)):
    qubit_operator = jordan_wigner(gsd_pool.operators[i]._f_operator)
    grad_obs = commutator(Hq, qubit_operator)
    qiskit_op = to_qiskit_operator(grad_obs)
    # print(f'Qubit Operator-{i}: {qubit_operator}')
    print(f'Grad Obs-{i}: {qiskit_op.paulis}')

Grad Obs-0: ['IXZX', 'IYZY', 'IXIX', 'IYIY', 'ZXZX', 'ZYZY', 'YZYZ', 'YIYI', 'XZXZ',
 'XIXI']
Grad Obs-1: ['XZXI', 'YZYI', 'XZXZ', 'YZYZ', 'XIXI', 'YIYI', 'ZXZX', 'IXIX', 'ZYZY',
 'IYIY']
Grad Obs-2: ['XZXI', 'XZXZ', 'YZYI', 'YZYZ', 'XIXI', 'XIXZ', 'YIYI', 'YIYZ', 'ZXZX',
 'IXIX', 'ZYZY', 'IYIY']
Grad Obs-3: ['IXZX', 'IXIX', 'IYZY', 'IYIY', 'ZXZX', 'ZXIX', 'ZYZY', 'ZYIY', 'YZYZ',
 'YIYI', 'XZXZ', 'XIXI']
Grad Obs-4: ['XYXY', 'XYYX', 'XXXX', 'XXYY', 'YYXX', 'YYYY', 'YXXY', 'YXYX', 'ZIII',
 'ZIZZ', 'ZZIZ', 'ZZZI', 'IIIZ', 'IIZI', 'IZII', 'IZZZ']
Grad Obs-5: ['ZIZZ', 'ZZIZ', 'ZIII', 'ZZZI', 'IIIZ', 'IZZZ', 'IIZI', 'IZII']
Grad Obs-6: ['IXZX', 'ZXZX', 'IYZY', 'ZYZY', 'IXIX', 'ZXIX', 'IYIY', 'ZYIY', 'YZYZ',
 'YIYI', 'XZXZ', 'XIXI']
Grad Obs-7: ['XZXI', 'XIXI', 'YZYI', 'YIYI', 'XZXZ', 'XIXZ', 'YZYZ', 'YIYZ', 'ZXZX',
 'IXIX', 'ZYZY', 'IYIY']


### Qubit Pool

In [25]:
qubit_pool = NoZPauliPool(molecule)

for k in range(len(qubit_pool.operators)):
    Ak = qubit_pool.operators[k]._q_operator
    # print(f'\nAk : Operator-{k}: {Ak}')
    
    grad_obs = commutator(Hq, Ak)
    # print(f'[H,Ak] : Grad Obs-{k}: {grad_obs}')

    grad_obs_qiskit = to_qiskit_operator(grad_obs)
    print(f'[H,Ak] : Grad Obs-{k}: {grad_obs_qiskit.paulis}')
    # print(f'\nOperator-{i}: {type(qubit_pool.operators[i]._f_operator)}')

[H,Ak] : Grad Obs-0: ['IXIX', 'IYIY', 'IXZX', 'ZXIX', 'IYZY', 'ZYIY']
[H,Ak] : Grad Obs-1: ['IYIY', 'IXIX', 'IYZY', 'ZYIY', 'IXZX', 'ZXIX']
[H,Ak] : Grad Obs-2: ['XIXI', 'YIYI', 'XIXZ', 'YIYZ', 'XZXI', 'YZYI']
[H,Ak] : Grad Obs-3: ['YIYI', 'XIXI', 'YIYZ', 'XIXZ', 'YZYI', 'XZXI']
[H,Ak] : Grad Obs-4: ['XXXX', 'XXYY', 'XYXY', 'YXXY', 'ZZIZ', 'IZZZ', 'ZIII', 'IIZI']
[H,Ak] : Grad Obs-5: ['XXYY', 'XXXX', 'XYYX', 'YXYX', 'ZZZI', 'IZII', 'ZIZZ', 'IIIZ']
[H,Ak] : Grad Obs-6: ['XYXY', 'XYYX', 'XXXX', 'YYXX', 'ZIII', 'IIZI', 'ZZIZ', 'IZZZ']
[H,Ak] : Grad Obs-7: ['YXXY', 'YXYX', 'YYXX', 'XXXX', 'IZII', 'ZZZI', 'IIIZ', 'ZIZZ']
[H,Ak] : Grad Obs-8: ['YYYY', 'YYXX', 'YXYX', 'XYYX', 'IIZI', 'ZIII', 'IZZZ', 'ZZIZ']
[H,Ak] : Grad Obs-9: ['YYXX', 'YYYY', 'YXXY', 'XYXY', 'IIIZ', 'ZIZZ', 'IZII', 'ZZZI']
[H,Ak] : Grad Obs-10: ['YXYX', 'YXXY', 'YYYY', 'XXYY', 'IZZZ', 'ZZIZ', 'IIZI', 'ZIII']
[H,Ak] : Grad Obs-11: ['XYYX', 'XYXY', 'XXYY', 'YYYY', 'ZIZZ', 'IIIZ', 'ZZZI', 'IZII']


### Qubit-Excitation Pool

In [31]:
qe_pool = QE(molecule)


for k in range(len(qe_pool.operators)):
    Ak = qe_pool.operators[k]._q_operator
    # print(f'\nAk : Operator-{k}: {Ak}')
    
    grad_obs = commutator(Hq, Ak)
    # print(f'[H,Ak] : Grad Obs-{k}: {grad_obs}')

    grad_obs_qiskit = to_qiskit_operator(grad_obs)
    print(f'[H,Ak] : Grad Obs-{k}: {grad_obs_qiskit.paulis}')
    # print(f'\nOperator-{i}: {type(qe_pool.operators[i]._f_operator)}')

[H,Ak] : Grad Obs-0: ['IXIX', 'IYIY', 'IXZX', 'IYZY', 'ZXIX', 'ZYIY']
[H,Ak] : Grad Obs-1: ['XIXI', 'YIYI', 'XIXZ', 'YIYZ', 'XZXI', 'YZYI']
[H,Ak] : Grad Obs-2: ['XXXX', 'YYXX', 'XYXY', 'YXXY', 'XYYX', 'YXYX', 'XXYY', 'YYYY', 'ZZIZ',
 'IIIZ', 'ZIII', 'IZII', 'ZIZZ', 'IZZZ', 'ZZZI', 'IIZI']
[H,Ak] : Grad Obs-3: ['ZIII', 'IIIZ', 'ZIZZ', 'IIZI', 'ZZIZ', 'IZII', 'ZZZI', 'IZZZ']
